Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

## Importing code

In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from IPython.display import display

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.model_selection import train_test_split

from timeit import default_timer as timer

import os

import pydotplus
from sklearn import tree
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from itertools import cycle


## Uploading data

In [13]:
# Upload dataset
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')


In [14]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [15]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [16]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.600,168.900
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.640,29.845
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.300,31.800
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.800,50.800
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.400,19.100


In [17]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [18]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [19]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

half_artworks = artworks.sample(frac=.5, random_state=2, axis=0)
quarter_artworks = artworks.sample(frac=.25, random_state=2, axis=0)
tenth_artworks = artworks.sample(frac=.1, random_state=2, axis=0)
onepercent_artworks = artworks.sample(frac=.01, random_state=2, axis=0)



In [28]:
working_artworks_df = tenth_artworks

# Final column drops and NA drop.
X = working_artworks_df.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(working_artworks_df.Artist)
nationalities = pd.get_dummies(working_artworks_df.Nationality)
dates = pd.get_dummies(working_artworks_df.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = working_artworks_df.Department

## Running the models

### 1000

In [31]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp1000 = MLPClassifier(hidden_layer_sizes=(1000,))
mlp1000.fit(X, Y)
print(' ')

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [32]:
mlp1000.score(X, Y)

0.7018477043673013

In [33]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [34]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp1000, X, Y, cv=5)

array([0.69058714, 0.61753731, 0.63695754, 0.51213819, 0.39093881])

In [35]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 500, 50

In [36]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp500_50 = MLPClassifier(hidden_layer_sizes=(500,50))
mlp500_50.fit(X, Y)
print(' ')

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [37]:
mlp500_50.score(X, Y)

0.5468458379992535

In [38]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [39]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp500_50, X, Y, cv=5)

array([0.49301025, 0.46735075, 0.71628558, 0.70401494, 0.6529659 ])

In [40]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 500, 50, 5

In [41]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp500_50_5 = MLPClassifier(hidden_layer_sizes=(500,50,5))
mlp500_50_5.fit(X, Y)
print(' ')

In [42]:
mlp500_50_5.score(X, Y)

0.6180477790220231

In [43]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [44]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp500_50_5, X, Y, cv=5)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.61742777, 0.61800373, 0.68082128, 0.61811391, 0.61840262])

In [45]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 100, 100, 10

In [46]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp100_100_10 = MLPClassifier(hidden_layer_sizes=(100,100,10))
mlp100_100_10.fit(X, Y)
print(' ')

In [47]:
mlp100_100_10.score(X, Y)

0.6180477790220231

In [48]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [49]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp100_100_10, X, Y, cv=5)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.61742777, 0.6693097 , 0.65328978, 0.64145658, 0.70994862])

In [50]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 500, 100, 10

In [51]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp500_100_10 = MLPClassifier(hidden_layer_sizes=(500,100,10))
mlp500_100_10.fit(X, Y)
print(' ')

In [52]:
mlp500_100_10.score(X, Y)

0.6180477790220231

In [53]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [54]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp500_100_10, X, Y, cv=5)

array([0.61742777, 0.61800373, 0.61829211, 0.61811391, 0.63521719])

In [55]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 500, 100, 10, 5

In [61]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp500_100_10_5 = MLPClassifier(hidden_layer_sizes=(500,100,10,5))
mlp500_100_10_5.fit(X, Y)
print(' ')

In [62]:
mlp500_100_10_5.score(X, Y)

0.6180477790220231

In [63]:
Y.value_counts()/len(Y)

Drawings & Prints       0.618
Photography             0.225
Architecture & Design   0.116
Painting & Sculpture    0.037
Media and Performance   0.004
Name: Department, dtype: float64

In [64]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp500_100_10_5, X, Y, cv=5)

array([0.61742777, 0.63386194, 0.63882408, 0.61811391, 0.61840262])

In [65]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 50, 25, 5, 1

In [66]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp50_25_5_1 = MLPClassifier(hidden_layer_sizes=(50,25,5,1))
mlp50_25_5_1.fit(X, Y)
print(' ')

In [67]:
mlp50_25_5_1.score(X, Y)

0.6180477790220231

In [68]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp50_25_5_1, X, Y, cv=5)

array([0.61742777, 0.61800373, 0.61829211, 0.61811391, 0.61840262])

In [69]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 25, 10, 5, 1

In [70]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp25_10_5_1 = MLPClassifier(hidden_layer_sizes=(25,10,5,1))
mlp25_10_5_1.fit(X, Y)
print(' ')

In [71]:
mlp25_10_5_1.score(X, Y)

0.6180477790220231

In [72]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp25_10_5_1, X, Y, cv=5)

array([0.61742777, 0.61800373, 0.61829211, 0.64379085, 0.61840262])

In [73]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




### 10, 5, 3, 1

In [74]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp10_5_3_1 = MLPClassifier(hidden_layer_sizes=(10, 5, 3, 1))
mlp10_5_3_1.fit(X, Y)
print(' ')

In [75]:
mlp10_5_3_1.score(X, Y)

0.6463232549458753

In [76]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp10_5_3_1, X, Y, cv=5)

array([0.61742777, 0.61800373, 0.65188987, 0.61811391, 0.61840262])

In [77]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




Aaaand there we go, now it's no longer working at "optimum"(?) accuracy.

### 25, 10, 5, 1

In [78]:
# Establish and fit the model, with a 500 and a 50 perceptron layers.
mlp25_10_5_1 = MLPClassifier(hidden_layer_sizes=(25,10,5,1), activation='logistic')
mlp25_10_5_1.fit(X, Y)
print(' ')

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [79]:
mlp25_10_5_1.score(X, Y)

0.6174878686076894

In [80]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp25_10_5_1, X, Y, cv=5)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.63373719, 0.61800373, 0.63369109, 0.63865546, 0.63148062])

In [81]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done


